# Initialize labeled dataset from CO3D
You can ignore this section if labeled dataset is already in S3 bucket (in which case skip to the next section).

In [1]:
# Download dataset

!cd .. && python -m scripts.dataset.labeled.download_labeled_dataset --category hydrant

Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/workspace/GSDiffusionModel/scripts/dataset/labeled/download_labeled_dataset.py", line 13, in <module>
    from tqdm import tqdm
ModuleNotFoundError: No module named 'tqdm'


In [10]:
# Convert dataset to GS

!cd .. && python -m scripts.dataset.labeled.create_gs_dataset --category hydrant --network_gui False

Directory structure verified. Found 1 category folders.
Processing category: hydrant
Processing folder: data/labeled_gs/raw/hydrant/hydrant
Generating GS for 134_15449_31106
/workspace/GSDiffusionModel/submodules/gaussian-splatting/train.py
/workspace/GSDiffusionModel/data/labeled_gs/raw/hydrant/hydrant/134_15449_31106
Generating GS for 216_22871_50171
/workspace/GSDiffusionModel/submodules/gaussian-splatting/train.py
/workspace/GSDiffusionModel/data/labeled_gs/raw/hydrant/hydrant/216_22871_50171
Generating GS for 206_21815_46194
/workspace/GSDiffusionModel/submodules/gaussian-splatting/train.py
/workspace/GSDiffusionModel/data/labeled_gs/raw/hydrant/hydrant/206_21815_46194
Generating GS for 216_22839_48459
/workspace/GSDiffusionModel/submodules/gaussian-splatting/train.py
/workspace/GSDiffusionModel/data/labeled_gs/raw/hydrant/hydrant/216_22839_48459
Error running script. Return code: 1
Error output:

Training progress:   6%|▌         | 300/5000 [00:09<01:50, 42.51it/s, Loss=0.0160375

In [5]:
!cd .. && python submodules/gaussian-splatting/render.py -m /workspace/GSDiffusionModel/data/labeled_gs/raw/hydrant/hydrant/106_12653_23216

Looking for config file in /workspace/GSDiffusionModel/data/labeled_gs/raw/hydrant/hydrant/106_12653_23216/cfg_args
Config file found: /workspace/GSDiffusionModel/data/labeled_gs/raw/hydrant/hydrant/106_12653_23216/cfg_args
Rendering /workspace/GSDiffusionModel/data/labeled_gs/raw/hydrant/hydrant/106_12653_23216
/workspace/GSDiffusionModel/data/labeled_gs/raw/hydrant/hydrant/106_12653_23216/point_cloud [01/09 11:09:15]
['iteration_5000', 'iteration_2000', 'iteration_10'] [01/09 11:09:15]
Loading trained model at iteration 5000 [01/09 11:09:15]
Reading camera 202/202 [01/09 11:09:15]
Loading Training Cameras [01/09 11:09:15]
Loading Test Cameras [01/09 11:09:27]
Rendering progress: 100%|█████████████████████| 202/202 [00:42<00:00,  4.71it/s]
Rendering progress: 0it [00:00, ?it/s]


In [12]:
!cd .. && python submodules/gaussian-splatting/render.py -m /workspace/south-building

Looking for config file in /workspace/south-building/cfg_args
Config file found: /workspace/south-building/cfg_args
Rendering /workspace/south-building
/workspace/south-building/point_cloud [01/09 03:46:34]
['iteration_10'] [01/09 03:46:34]
Loading trained model at iteration 10 [01/09 03:46:34]
Reading camera 128/128 [01/09 03:46:34]
Loading Training Cameras [01/09 03:46:34]
[ INFO ] Encountered quite large input images (>1.6K pixels width), rescaling to 1.6K.
 If this is not desired, please explicitly specify '--resolution/-r' as 1 [01/09 03:46:34]
Loading Test Cameras [01/09 03:46:56]
Rendering progress: 100%|█████████████████████| 128/128 [02:00<00:00,  1.06it/s]
Rendering progress: 0it [00:00, ?it/s]


In [9]:
with open("/workspace/GSDiffusionModel/data/labeled_gs/raw/hydrant/hydrant/106_12653_23216/sparse/0/images.txt", "r") as f:
    lines = f.read().split('\n')
    print('\n'.join(lines[0:5]))
    print(lines[98])

# Image list with two lines of data per image:
#   IMAGE_ID, QW, QX, QY, QZ, TX, TY, TZ, CAMERA_ID, NAME
#   POINTS2D[] as (X, Y, POINT3D_ID)
# Number of images: 202, mean observations per image: 2500
1 0.43923069735190345 -0.5312714836975936 -0.012955058327317137 -0.7243336051690912 -0.010589353740215302 2.3088395595550537 9.083442687988281 1 frame000001.jpg
48 0.06198152213049953 -0.05356958804565001 0.010805305963446976 0.9965800697957199 0.38255879282951355 0.7826452851295471 7.525020122528076 48 frame000048.jpg


In [4]:
with open("/workspace/GSDiffusionModel/data/labeled_gs/raw/hydrant/hydrant/106_12653_23216/sparse/0/cameras.txt", "r") as f:
    lines = f.read().split('\n')
    print('\n'.join(lines[0:5]))
    print(lines[50])

# Camera list with one line of data per camera:
#   CAMERA_ID, MODEL, WIDTH, HEIGHT, PARAMS[6]
# Number of cameras: 202
1 PINHOLE 719 1646 247.01188123953466 565.4819979419667 359 823
2 PINHOLE 719 1646 246.9641622844523 565.3727553827656 359 823
48 PINHOLE 719 1646 244.61185364354063 559.9876371311098 359 823


In [20]:
with open("/workspace/GSDiffusionModel/data/labeled_gs/raw/hydrant/hydrant/106_12653_23216/sparse/0/points3D.txt", "r") as f:
    lines = f.read().splitlines()  # Use splitlines to handle newlines correctly
    print(lines[:5])

    x_range = [float('inf'), float('-inf')]
    y_range = [float('inf'), float('-inf')]
    z_range = [float('inf'), float('-inf')]

    for line in lines[3:]:
        if line.strip():  # Skip empty lines
            segs = line.split(' ')

            # Convert string values to floats
            x = float(segs[1])
            y = float(segs[2])
            z = float(segs[3])

            # Update x_range
            if x < x_range[0]:
                x_range[0] = x
            if x > x_range[1]:
                x_range[1] = x

            # Update y_range
            if y < y_range[0]:
                y_range[0] = y
            if y > y_range[1]:
                y_range[1] = y

            # Update z_range
            if z < z_range[0]:
                z_range[0] = z
            if z > z_range[1]:
                z_range[1] = z

    print("X Range:", x_range)
    print("Y Range:", y_range)
    print("Z Range:", z_range)

['# 3D point list with one line of data per point:', '#   POINT3D_ID, X, Y, Z, R, G, B, ERROR, TRACK[] as (IMAGE_ID, POINT2D_IDX)', '# Number of points: 980001, mean track length: 0.0', '1 -0.4043790102005005 -1.2760553359985352 -0.4534003734588623 156 161 161 0.0', '2 0.3273124694824219 -0.9985339641571045 0.8547940254211426 220 220 230 0.0']
X Range: [-1.280186414718628, 1.9200682640075684]
Y Range: [-1.9406838417053223, 3.504551887512207]
Z Range: [-1.3880338668823242, 3.6833600997924805]


In [18]:
with open("/workspace/GSDiffusionModel/data/labeled_gs/raw/hydrant/hydrant/106_12653_23216/sparse/0/images.txt", "r") as f:
    lines = f.read().splitlines()  # Use splitlines to handle newlines correctly
    print(lines[:5])

    x_range = [float('inf'), float('-inf')]
    y_range = [float('inf'), float('-inf')]
    z_range = [float('inf'), float('-inf')]

    for line in lines[4:]:
        if line.strip():  # Skip empty lines
            segs = line.split(' ')

            # Convert string values to floats
            x = float(segs[5])
            y = float(segs[6])
            z = float(segs[7])

            # Update x_range
            if x < x_range[0]:
                x_range[0] = x
            if x > x_range[1]:
                x_range[1] = x

            # Update y_range
            if y < y_range[0]:
                y_range[0] = y
            if y > y_range[1]:
                y_range[1] = y

            # Update z_range
            if z < z_range[0]:
                z_range[0] = z
            if z > z_range[1]:
                z_range[1] = z

    print("X Range:", x_range)
    print("Y Range:", y_range)
    print("Z Range:", z_range)
    

['# Image list with two lines of data per image:', '#   IMAGE_ID, QW, QX, QY, QZ, TX, TY, TZ, CAMERA_ID, NAME', '#   POINTS2D[] as (X, Y, POINT3D_ID)', '# Number of images: 202, mean observations per image: 2500', '1 0.7243336051690914 0.012955058327316998 -0.531271483697594 -0.43923069735190345 -0.010589353740215302 2.3088395595550537 9.083442687988281 1 frame000001.jpg']
X Range: [-1.2386572360992432, 0.781455934047699]
Y Range: [-0.30876585841178894, 2.3088395595550537]
Z Range: [6.0859150886535645, 9.089887619018555]


### Testing CO3D Data

In [15]:
def SH2RGB(sh):
    return sh * 0.28209479177387814 + 0.5

print(SH2RGB(0.39619562)*255) 
print(SH2RGB(0.4657036)* 255) 
print(SH2RGB(0.4657036)* 255) 

156.00000383603373
161.0000028179381
161.0000028179381


In [20]:
from plyfile import PlyData

# Load the PLY file
ply_data = PlyData.read('/workspace/GSDiffusionModel/data/labeled_gs/raw/hydrant/hydrant/106_12653_23216/point_cloud/iteration_10/point_cloud.ply')  # Replace with your PLY file path

# Access the vertex data (assuming the file contains vertices)
vertex_data = ply_data['vertex']

# Print a few lines of the vertex data
for i in range(5):  # Adjust the range to view more or fewer lines
    print(vertex_data[i])
print(ply_data)

(-0.404379, -1.2760553, -0.45340037, 0., 0., 0., 0.39619562, 0.4657036, 0.4657036, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -2.1972246, -5.257743, -5.257743, -5.257743, 1., 0., 0., 0.)
(0.32731247, -0.99853396, 0.854794, 0., 0., 0., 1.2858979, 1.2858979, 1.4249139, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -2.1972246, -5.399721, -5.399721, -5.399721, 1., 0., 0., 0.)
(0.49003685, -0.7794697, -0.8508241, 0., 0., 0., 1.2997994, 1.2580947, 1.2580947, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -2.1972246, -4.3777213, -4.3777213, -4.3777213, 1., 0., 0., 0.)
(-0.60084534, 1.0265565, 0.2

In [21]:
import numpy as np
from plyfile import PlyData, PlyElement

new_data = np.array(
    [(vertex[0], vertex[1], vertex[2], SH2RGB(vertex[6])*255, SH2RGB(vertex[7])*255, SH2RGB(vertex[8])*255) for vertex in vertex_data],
    dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4'), ('red', 'u1'), ('green', 'u1'), ('blue', 'u1')]
)

# Create the PLY element
vertex_element = PlyElement.describe(new_data, 'vertex')

# Write to PLY file
PlyData([vertex_element]).write('/workspace/GSDiffusionModel/data/labeled_gs/raw/hydrant/hydrant/106_12653_23216/point_cloud/iteration_10/output.ply')

In [3]:
from plyfile import PlyData

# Load the PLY file
ply_data = PlyData.read('/workspace/south-building/point_cloud/iteration_10/point_cloud.ply')  # Replace with your PLY file path

# Access the vertex data (assuming the file contains vertices)
vertex_data = ply_data['vertex']

# Print a few lines of the vertex data
for i in range(5):  # Adjust the range to view more or fewer lines
    print(vertex_data[i])
print(ply_data)

PlyHeaderParseError: line 1: expected 'ply'

In [1]:
from typing import List
from co3d.dataset.data_types import (
    load_dataclass_jgzip, FrameAnnotation, SequenceAnnotation
)

category_frame_annotations = load_dataclass_jgzip(
    f"../data/labeled_gs/raw/hydrant/hydrant/frame_annotations.jgz", List[FrameAnnotation]
)
category_sequence_annotations = load_dataclass_jgzip(
    f"../data/labeled_gs/raw/hydrant/hydrant/sequence_annotations.jgz", List[SequenceAnnotation]
)

In [3]:
category_frame_annotations[0]

FrameAnnotation(sequence_name='605_94563_187702', frame_number=6, frame_timestamp=0.3845771144278607, image=ImageAnnotation(path='hydrant/605_94563_187702/images/frame000006.jpg', size=(1268, 713)), depth=DepthAnnotation(path='hydrant/605_94563_187702/depths/frame000006.jpg.geometric.png', scale_adjustment=1.0, mask_path='hydrant/605_94563_187702/depth_masks/frame000006.png'), mask=MaskAnnotation(path='hydrant/605_94563_187702/masks/frame000006.png', mass=1000000), viewpoint=ViewpointAnnotation(R=((-0.9998694658279419, -0.001025464036501944, -0.016123276203870773), (0.0009754493949003518, -0.999994695186615, 0.003109571523964405), (-0.016126379370689392, 0.003093438223004341, 0.9998651742935181)), T=(-0.06160982325673103, 1.5896116495132446, 14.91888427734375), focal_length=(3.4211699962615967, 3.4211699962615967), principal_point=(0.001402524532750249, -0.001402524532750249), intrinsics_format='ndc_isotropic'), meta={'frame_type': 'test_unseen', 'frame_splits': ['singlesequence_hydran

### Upload labeled dataset to HuggingFace


In [8]:
!cd .. && python -m scripts.dataset.labeled.upload_to_hf --category hydrant

134_15449_31106
             x         y         z  ...     rot_1     rot_2     rot_3
0    -0.500877  2.500373  2.824118  ...  0.334788  0.025459 -0.072700
1     0.362021  1.431359  0.213970  ...  0.009734  0.133791 -0.261926
2     0.950835  0.191075 -0.093043  ... -0.088824 -0.008922  0.137166
3    -0.960510 -0.354492 -0.127541  ...  0.157182  0.235684 -0.018999
4     0.715609 -0.452055  1.929193  ... -0.111809 -0.174941  0.188118
...        ...       ...       ...  ...       ...       ...       ...
2043 -1.182361 -0.520987  1.062389  ...  0.126196  0.292599  0.255409
2044  0.242414 -0.862372 -1.938615  ...  0.021236 -0.025199 -0.001159
2045  0.366704 -0.703431 -1.251408  ...  0.035494  0.271910 -0.301101
2046  0.164336 -0.841585  0.283777  ... -0.179888 -0.071808  0.172398
2047 -0.176113 -0.427542 -1.544789  ... -0.285841  0.078026 -0.026266

[2048 rows x 14 columns]
Dataset({
    features: ['x', 'y', 'z', 'f_dc_0', 'f_dc_1', 'f_dc_2', 'opacity', 'scale_0', 'scale_1', 'scale_2', 'rot

# Download labeled dataset from S3 bucket


In [4]:
!cd .. && ls -la

total 11801
drwxrwxrwx 7 root root 3003630 Aug 30 22:34 .
drwxrwxrwx 4 root root 3003630 Aug 30 21:56 ..
drwxrwxrwx 8 root root 1004083 Aug 30 21:55 .git
-rw-rw-rw- 1 root root      49 Aug 30 21:55 .gitignore
drwxrwxrwx 2 root root       1 Aug 30 22:34 .ipynb_checkpoints
-rw-rw-rw- 1 root root    2004 Aug 30 21:55 README.md
drwxrwxrwx 3 root root 3003630 Aug 30 21:55 data
drwxrwxrwx 3 root root 1062694 Aug 30 22:36 notebooks
-rw-rw-rw- 1 root root      13 Aug 30 21:55 requirements.txt
drwxrwxrwx 3 root root 1000680 Aug 30 22:19 scripts
